In [56]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
import torchvision.models as models
import torch.nn as nn


In [57]:
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import os
from PIL import Image
import tqdm


class CatNotCatDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (string): Directory with all the animal subdirectories.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []

        # Separate handling for 'cat' and 'not cat' images
        for sub_dir in os.listdir(root_dir):
            class_path = os.path.join(root_dir, sub_dir)
            if os.path.isdir(class_path):
                # Label 1 for cat, 0 for not cat
                label = 1 if sub_dir.lower() == 'cat' else 0
                # If 'not cat', traverse further into subdirectories
                if label == 0:
                    for sub_class in os.listdir(class_path):
                        sub_class_path = os.path.join(class_path, sub_class)
                        if os.path.isdir(sub_class_path):
                            for img_file in os.listdir(sub_class_path):
                                if img_file.lower().endswith(('png', 'jpg', 'jpeg')):
                                    img_path = os.path.join(sub_class_path, img_file)
                                    self.samples.append((img_path, label))
                else:
                    for img_file in os.listdir(class_path):
                        if img_file.lower().endswith(('png', 'jpg', 'jpeg')):
                            img_path = os.path.join(class_path, img_file)
                            self.samples.append((img_path, label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        return image, label


In [58]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # ImageNet normalization
])

dataset = CatNotCatDataset(root_dir='animals', transform=transform)

# Splitting dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Creating data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [59]:
# Define the ResNet-18 model class for binary classification
class ResNet18Binary(nn.Module):
    def __init__(self, pretrained=True):
        super(ResNet18Binary, self).__init__()
        self.resnet18 = models.resnet18(pretrained=pretrained)
        num_ftrs = self.resnet18.fc.in_features
        self.resnet18.fc = nn.Linear(num_ftrs, 2)

    def forward(self, x):
        return self.resnet18(x)

In [60]:
import matplotlib.pyplot as plt

def visualize_feature_maps(feature_maps):
    # Assuming feature_maps is a list of tensors from the convolutional layers
    for layer, f_map in enumerate(feature_maps):
        layer += 1  # Start layer indexing at 1
        plt.figure(figsize=(20, 15))
        channels = f_map.shape[1]
        
        for i in range(channels):
            plt.subplot(channels // 8 + 1, 8, i + 1)  # Arrange plots in a grid
            plt.imshow(f_map[0, i].detach().cpu().numpy(), cmap='gray')
            plt.axis('off')
        
        plt.show()


In [61]:
import torch
# Function to train the model
def train_model(model, train_loader, val_loader, num_epochs=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    
    for epoch in tqdm.tqdm(range(num_epochs)):
        model.train()
        running_loss = 0.0
        for images, labels in tqdm.tqdm(train_loader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        # Print accuracy for the current epoch
        print(f'Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}, ' +
              f'Validation Loss: {val_loss / len(val_loader)}, ' +
              f'Accuracy: {100 * correct / total}%')

    print('Finished Training')

model = ResNet18Binary(pretrained=True)
train_model(model, train_loader, val_loader, num_epochs=3)

 33%|███▎      | 1/3 [00:05<00:10,  5.20s/it]

Epoch 1, Loss: 0.4653880000114441, Validation Loss: 0.6210405230522156, Accuracy: 66.66666666666667%


 67%|██████▋   | 2/3 [00:10<00:05,  5.22s/it]

Epoch 2, Loss: 0.4340703785419464, Validation Loss: 0.47446754574775696, Accuracy: 83.33333333333333%


100%|██████████| 3/3 [00:15<00:00,  5.09s/it]

Epoch 3, Loss: 0.24981248378753662, Validation Loss: 0.3207429349422455, Accuracy: 91.66666666666667%
Finished Training
